In [19]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import requests
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub 
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
search_tool = DuckDuckGoSearchRun()

result = search_tool.invoke("PSL Match NEWS today")
result

"PZ vs IU: Where to watch live streaming of today's PSL match? A Sports and PTV Sports will broadcast the match live in Pakistan, while live streaming will be available on the following platforms: Walee Technologies, Tapmad, Tamasha, Daraz, Myco and Begin.. PSL matches are not being live streamed in India currently. You can find the complete live streaming details for PSL 2025 here. Islamabad United won the toss and opted to bat first against Peshawar Zalmi in the 22nd match of the PSL 10 in Lahore on Friday. Close 30.9 C ... ARY NEWS brings you 24/7 Live Streaming, Headlines ... Lahore Qalandars face Islamabad United in the 19th match of the Pakistan Super League 2025 at the Gaddafi Stadium, Lahore tonight. Both teams are enjoying a successful season so far. ISL skipper Shadab Khan won the toss and elected to bowl first on the night. Shadab Khan-led Islamabad United have ... Islamabad United captain Shadab Khan and Peshawar Zalmi skipper Babar Azam at toss ceremony during their match 

In [ ]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=<YOUR_API_KEY>&query={city}'

  response = requests.get(url)

  return response.json()


In [31]:
llm = ChatGroq(model="llama3-8b-8192")


In [32]:
llm.invoke("Hi! How are you?")


AIMessage(content="I'm just a language model, I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you may have! How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 16, 'total_tokens': 61, 'completion_time': 0.0375, 'prompt_time': 0.00460468, 'queue_time': 0.01719799, 'total_time': 0.04210468}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-c20c68ea-e4e9-4b96-a896-a89991f733d1-0', usage_metadata={'input_tokens': 16, 'output_tokens': 45, 'total_tokens': 61})

In [33]:
# step 2: Pull the React prompt from LangChain Hub
prompt = hub.pull("hwchase17/react") # Reasoning + action


In [34]:
# step 3: Create the ReAct agent manually with the pulled prompt

agent = create_react_agent(
    llm = llm,
    tools = [search_tool, get_weather_data],
    prompt = prompt
)

In [35]:
# step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent = agent,
    tools = [search_tool, get_weather_data],
    verbose = True
)

In [36]:
# step 5: invoke
response = agent_executor.invoke({'input': "What is the latest news about PSL Match today? and What's the weather condition of Lahore?"})



> Entering new AgentExecutor chain...


Thought: I need to get the latest news about PSL Match today and the weather condition of Lahore.

Action: duckduckgo_search
Action Input: "PSL Match today news"Karachi Kings are taking on Multan Sultans while Lahore Qalandars will face off against Quetta Gladiators in today's PSL 2025 matches - you can follow the live scores, match updates, and live streaming details for the KK vs MS, and LQ vs QG games here. As the Pakistan Super League 2025 enters its final stages, the race for playoff spots has intensified. Match 21 has the Lahore Qalandars take on Quetta Gladiators at the Gaddafi Stadium in Lahore. Quetta skipper Saud Shakeel won the toss and opted to bowl first. Both teams come into this clash on ... Islamabad United won the toss and opted to bat first against Peshawar Zalmi in the 22nd match of the PSL 10 in Lahore on Friday. Close 30.9 C ... ARY NEWS brings you 24/7 Live Streaming, Headlines ... Today's PSL match: All you need to know. Match: Multan Sultans vs Quetta Gladiators

In [37]:
print(response)

{'input': "What is the latest news about PSL Match today? and What's the weather condition of Lahore?", 'output': "Karachi Kings are taking on Multan Sultans while Lahore Qalandars will face off against Quetta Gladiators in today's PSL 2025 matches. The weather condition in Lahore is partly cloudy with a temperature of 25 degrees Celsius."}
